In [ ]:
source("diabetes_analysis_v07.R")

# Treg HPAP

In [ ]:
treg_hpap_spleen  <- readRDS("../data/published_data/HPAP/treg_spleen.rds")

In [ ]:
DimPlot(treg_hpap_spleen)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(treg_hpap_spleen, group.by = "seurat_clusters2", 
       cols = c("#f8766dff","#c77cffff","#00bfc4ff","#99c03aff"))

## Density

In [ ]:
density  <- treg_hpap_spleen@meta.data  %>% dplyr::select(Patient_ID, Disease)

density$x_umap  <- treg_hpap_spleen@reductions$umap@cell.embeddings[,1]
density$y_umap  <- treg_hpap_spleen@reductions$umap@cell.embeddings[,2]


library(ggplot2)
library(dplyr)
library(viridis)

colfunc <- colorRampPalette(c("white", "grey85","grey75","dodgerblue","green","yellow","red"))


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)

p1  <- density %>%
filter(Disease == "T1DM")  %>% 
  ggplot(aes(x=x_umap, y= y_umap)) + 
 geom_point(shape = "")  + geom_density_2d_filled(bins = 30)  + 
theme_classic() + ggtheme() +
xlim(-10,10)+  ylim(-5,5) +
scale_fill_manual(values = colfunc(30))  + 
ggtitle("T1DM") + NoLegend()

p2   <-   density %>%
filter(Disease == "Ctrl")  %>% 
  ggplot(aes(x=x_umap, y= y_umap)) + 
 geom_point(shape = "")  + geom_density_2d_filled(bins = 30)  + 
theme_classic() + ggtheme() +
xlim(-10,10)+  ylim(-5,5) +
scale_fill_manual(values = colfunc(30))  + 
ggtitle("Ctrl") + NoLegend()


p1 + p2

## Cluster quantification

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)

seurat_meta_data <- treg_hpap_spleen@meta.data
seurat_meta_data$sample  <- seurat_meta_data$source
seurat_meta_data$sample  <- paste(seurat_meta_data$Patient_ID, seurat_meta_data$Tissue) 

# Create grouped dataframe, calculate the frequencies of clusters
df4 <- seurat_meta_data %>% group_by(sample, seurat_clusters2) %>% 
  summarise(n = n()) %>% 
  unique() %>% 
  mutate(freq = n / sum(n)) 


# As we've lost non-grouping variables, let's join them back
md_to_join <- seurat_meta_data %>% dplyr::select(sample, Disease, Patient_ID, Tissue) %>% unique()
df4  <- left_join(df4, md_to_join, by = "sample")

# The final plot

df4 %>% ggplot(aes(x = Disease, y = freq*100)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, alpha = 0.7, aes(fill = Disease)) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
  geom_jitter(position=position_jitter(0.05), size = 3, aes(fill = Disease), shape = 21) + # in aes, you can also use shape or fill (for the shapes that allow it)
  facet_wrap(~seurat_clusters2, scales = "free", ncol = 4) +
scale_color_manual(values = c("dodgerblue","indianred2")) +
scale_fill_manual(values = c("dodgerblue","indianred2")) +

  ylab("Frequency") +
  xlab("Condition") +
ggpubr::stat_compare_means(label = "p.format", size = 5) +
  ylim(c(0,NA)) + # This ensures that our x axis starts at zero, but feel free to remove this line
  theme_classic() +
  theme(strip.background = element_blank(), panel.grid = element_blank()) + NoLegend() +ggtheme() 


In [ ]:

df4 %>% 
mutate(Disease = ifelse(Disease == "Ctrl", "Ctrl","Dia"))  %>% 
ggplot(aes(x = Disease, y = freq*100)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, alpha = 0.3, aes(fill = Disease)) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
geom_beeswarm(size = 3, cex = 3, 
              color = "black", method = "center",
             aes(fill = Disease), shape = 21) +
facet_wrap(~seurat_clusters2, scales = "free", ncol = 5) +
   scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
  ylab("Frequency") +
  xlab("Condition") +
#ggpubr::stat_compare_means() +
  ylim(c(0,NA)) + # This ensures that our x axis starts at zero, but feel free to remove this line
  theme_classic() +
  theme(strip.background = element_blank(), panel.grid = element_blank()) + 
ggtheme()

ggsave("../../240617_VN_Diabetes_V06/figures/treg_validation/hpap_treg_subsets_in_condition.svg",
       width = 16, height = 8, units = "cm")

In [ ]:
df4  %>% group_by(seurat_clusters2, Disease)  %>% 
summarise(mean_freq = median(freq))

In [ ]:
df4_mean_hpap  <- df4  %>% group_by(seurat_clusters2, Disease)  %>% 
summarise(median_freq = median(freq))

In [ ]:
df4_mean_hpap$dataset  <- "HPAP"

## Heatmap

In [ ]:
genes_treg  <-  c("TCF7","LEF1","NELL2","IL7R","IL4R","GZMK","CD226",
    "CCR4","CTLA4","TNFRSF9","FOXP3","IL2RA")

In [ ]:
avgexp = AverageExpression(treg_hpap_spleen, features = genes_treg, return.seurat = F, 
                           group.by = "seurat_clusters2", assays = "RNA")

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3)
pheatmap(t(avgexp$RNA[match(c("TCF7","LEF1","NELL2","IL7R","IL4R","GZMK","CD226",
    "CCR4","CTLA4","TNFRSF9","FOXP3","IL2RA"),
rownames(avgexp$RNA)),]), main = "", scale = "column", cluster_cols = F, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
pheatmap(t(avgexp$RNA[match(c("TCF7","LEF1","NELL2","IL7R","IL4R","GZMK","CD226",
    "CCR4","CTLA4","TNFRSF9","FOXP3","IL2RA"),
rownames(avgexp$RNA)),]), main = "", scale = "column", cluster_cols = F, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 5, height = 3.3,
                  fontsize = 9, filename = "../../240617_VN_Diabetes_V06/figures/treg_validation/hpap_heatmap.pdf")

# Treg ParseBio

In [ ]:
treg_pb  <- readRDS("../../../DATA_scRNAseq//Analysis of previously published data/081_ParseBio_1M_Diabetes/241020_ParseBio_200k_treg.rds")

options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(treg_pb)

In [ ]:
treg_pb@meta.data  <- treg_pb@meta.data  %>% mutate(seurat_clusters2 = recode(seurat_clusters, 
                                                                                               "0" = "3",
                                                                                               "1" = "1",
                                                                                               "2" = "2",
                                                                                               "3" = "0"))

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(treg_pb, label = F, group.by = "seurat_clusters2", 
       cols = c("#f8766dff","#00bfc4ff","#99c03aff","#c77cffff"))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4)

seurat_meta_data <- treg_pb@meta.data

# Create grouped dataframe, calculate the frequencies of clusters
df4 <- seurat_meta_data %>% group_by(sample, seurat_clusters2) %>% 
  summarise(n = n()) %>% 
  unique() %>% 
  mutate(freq = n / sum(n)) 


# As we've lost non-grouping variables, let's join them back
md_to_join <- seurat_meta_data %>% dplyr::select(sample, Disease) %>% unique()
df4  <- left_join(df4, md_to_join, by = "sample")

# The final plot

df4 %>% ggplot(aes(x = Disease, y = freq)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
  geom_jitter(position=position_jitter(0.2), size = 2, aes(color = Disease)) + # in aes, you can also use shape or fill (for the shapes that allow it)
  facet_wrap(~seurat_clusters2, scales = "free", ncol = 6) +
  ylab("Frequency") +
  xlab("Condition") +
ggpubr::stat_compare_means() +
  ylim(c(0,NA)) + # This ensures that our x axis starts at zero, but feel free to remove this line
  theme_classic() +
  theme(strip.background = element_blank(), panel.grid = element_blank())


In [ ]:

df4 %>% 
mutate(Disease = ifelse(Disease == "H", "Ctrl","Dia"))  %>% 
ggplot(aes(x = Disease, y = freq*100)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, alpha = 0.3, aes(fill = Disease)) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
geom_beeswarm(size = 3, cex = 3, 
              color = "black", method = "center",
             aes(fill = Disease), shape = 21) +
facet_wrap(~seurat_clusters2, scales = "free", ncol = 5) +
   scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
  ylab("Frequency") +
  xlab("Condition") +
#ggpubr::stat_compare_means() +
  ylim(c(0,NA)) + # This ensures that our x axis starts at zero, but feel free to remove this line
  theme_classic() +
  theme(strip.background = element_blank(), panel.grid = element_blank()) + 
ggtheme()



In [ ]:
ggsave("../figures/treg_validation/pb_treg_subsets_in_condition.svg",
       width = 16, height = 8, units = "cm")

In [ ]:
df4  %>% group_by(seurat_clusters2, Disease)  %>% 
summarise(median_freq = median(freq))

In [ ]:
df4_mean_pb  <- df4  %>% group_by(seurat_clusters2, Disease)  %>% 
summarise(median_freq = median(freq))  %>% mutate(Disease = ifelse(Disease == "D", "Dia","Ctrl"))

In [ ]:
df4_mean_pb$dataset  <- "PB"

### Heatmap

In [ ]:
avgexp = AverageExpression(treg_pb, features = rev(str_to_upper(c(
    "TCF7","LEF1","NELL2","IL7R","IL4R","GZMK","CD226",
    "CCR4","CTLA4","TNFRSF9","FOXP3","IL2RA"   
    ))), return.seurat = F, 
                           group.by = "seurat_clusters2", assays = "RNA")

avgexp

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3)
pheatmap(t(avgexp$RNA[match(c("TCF7","LEF1","NELL2","IL7R","IL4R","GZMK","CD226",
    "CCR4","CTLA4","TNFRSF9","FOXP3","IL2RA"),
rownames(avgexp$RNA)),]), main = "", scale = "column", cluster_cols = F, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 9, height = 3.3,
                  fontsize = 9)

In [ ]:
pheatmap(t(avgexp$RNA[match(c("TCF7","LEF1","NELL2","IL7R","IL4R","GZMK","CD226",
    "CCR4","CTLA4","TNFRSF9","FOXP3","IL2RA"),
rownames(avgexp$RNA)),]), main = "", scale = "column", cluster_cols = F, cluster_rows = F,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white", width = 5, height = 3.3,
                  fontsize = 9, filename = "../figures/treg_validation/pb_heatmap.pdf")

### Density

In [ ]:
density  <- treg@meta.data  %>% dplyr::select(sample, Disease)

density$x_umap  <- treg@reductions$umap@cell.embeddings[,1]
density$y_umap  <- treg@reductions$umap@cell.embeddings[,2]


library(ggplot2)
library(dplyr)
library(viridis)

colfunc <- colorRampPalette(c("white", "grey85","grey75","dodgerblue","green","yellow","red"))


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)

p1  <- density %>%
filter(Disease == "D")  %>% 
  ggplot(aes(x=x_umap, y= y_umap)) + 
 geom_point(shape = "")  + geom_density_2d_filled(bins = 30)  + 
theme_classic() + ggtheme() +
xlim(-8,8)+  ylim(-6,6) +
scale_fill_manual(values = colfunc(30))  + 
ggtitle("Dia T0") + NoLegend()

p2   <-   density %>%
filter(Disease == "H")  %>% 
  ggplot(aes(x=x_umap, y= y_umap)) + 
 geom_point(shape = "")  + geom_density_2d_filled(bins = 30)  + 
theme_classic() + ggtheme() +
xlim(-8,8)+  ylim(-6,6) +
scale_fill_manual(values = colfunc(30))  + 
ggtitle("Ctrl T0") + NoLegend()


p1 + p2

# Treg NatCom

In [ ]:
treg_nc  <- readRDS("../../../DATA_scRNAseq//Analysis of previously published data/080_Zhong_NatCom_Diabetes/241019_NatComm_merged_filt_treg_noPR.rds")

In [ ]:
Idents(treg_nc)  <- treg_nc$seurat_clusters

In [ ]:
DimPlot(treg_nc)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(treg_nc, label = F, group.by = "seurat_clusters", 
       cols = c("#c77cffff","#00bfc4ff","#99c03aff","#f8766dff"))

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 4)

seurat_meta_data <- treg_nc@meta.data
seurat_meta_data$sample  <- seurat_meta_data$source

# Create grouped dataframe, calculate the frequencies of clusters
df4 <- seurat_meta_data %>% group_by(sample, seurat_clusters) %>% 
  summarise(n = n()) %>% 
  unique() %>% 
  mutate(freq = n / sum(n)) 


# As we've lost non-grouping variables, let's join them back
md_to_join <- seurat_meta_data %>% dplyr::select(sample, Disease) %>% unique()
df4  <- left_join(df4, md_to_join, by = "sample")

In [ ]:
# The final plot

df4 %>% 
mutate(Disease = ifelse(Disease == "N", "Ctrl","Dia"))  %>% 
ggplot(aes(x = Disease, y = freq*100)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, alpha = 0.3, aes(fill = Disease)) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
geom_beeswarm(size = 3, cex = 3, 
              color = "black", method = "center",
             aes(fill = Disease), shape = 21) +
facet_wrap(~seurat_clusters, scales = "free", ncol = 5) +
   scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
  ylab("Frequency") +
  xlab("Condition") +
#ggpubr::stat_compare_means() +
  ylim(c(0,NA)) + # This ensures that our x axis starts at zero, but feel free to remove this line
  theme_classic() +
  theme(strip.background = element_blank(), panel.grid = element_blank()) + 
ggtheme()



In [ ]:


df4_mean_nc  <- df4  %>% group_by(seurat_clusters, Disease)  %>% 
summarise(median_freq = median(freq))  %>% 
mutate(Disease = ifelse(Disease == "N", "Ctrl","Dia")) 


df4_mean_nc$dataset  <- "NC"



In [ ]:
df4_mean_nc$seurat_clusters2  <- df4_mean_nc$seurat_clusters

In [ ]:
df4_mean_nc$seurat_clusters  <- NULL

In [ ]:
df4_mean_nc

## Cross dataset comparison

In [ ]:
df_cross  <- rbind(df4_mean_hpap, df4_mean_nc, df4_mean_pb)

In [ ]:
df_cross

In [ ]:
df_cross  %>% 
mutate(Disease = gsub(Disease, pattern = "T1DM", replacement = "Dia"))  %>% 
ggplot(aes(x = Disease, y = median_freq*100)) + # you can change the x to whatever variable you're interested in
#  geom_boxplot(outlier.shape = NA, alpha = 0.3, aes(fill = Disease)) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
geom_beeswarm(size = 3, cex = 3, 
              color = "black", method = "center",
             aes(fill = Disease), shape = 21) +
geom_line(aes(group = dataset, alpha = 0.1)) +
facet_wrap(~seurat_clusters2, scales = "free", ncol = 5) +
   scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
  ylab("Frequency") +
  xlab("Condition") +
#ggpubr::stat_compare_means() +
  ylim(c(0,NA)) + # This ensures that our x axis starts at zero, but feel free to remove this line
  theme_classic() +
  theme(strip.background = element_blank(), panel.grid = element_blank()) + 
ggtheme()

## With our Treg

In [ ]:
treg  <- readRDS("../../240617_VN_Diabetes_V06/data/processed/L3/cd4_l3_treg.rds")

In [ ]:
treg_subset  <- subset(treg, Condition %in% c("Ctrl T0", "Dia T0", "Dia T1") & Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20")) 

df4  <- create_df4(treg_subset)

df4

 p5  <-   df4  %>% 
filter(Experiment_ID %in% c("Exp16", "Exp18", "Exp19", "Exp20"))  %>% 
  ggplot(aes(x = Condition, y = freq*100)) + # you can change the x to whatever variable you're interested in
   geom_violin(alpha = 0.3, aes(fill = Condition)) +
  stat_summary(fun = "median",
               geom = "crossbar", 
               width = 0.75,
               color = "grey30") +
   scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
geom_beeswarm(size = 3, cex = 3, 
              color = "black", method = "center",
             aes(fill = Condition), shape = 21) +
   facet_wrap(~annotations_manual, scales = "free", ncol = 9) +
  ylab("Frequency") +
  xlab("Condition") +
  theme_classic() +
ggtheme() +
ggpubr::stat_compare_means(label.x= 1.5, 
                           size = 7, vjust = -1, label = "p.format", comparisons = list(c(1,2)))+
ggtheme() +
 scale_y_continuous(limits = c(0,NA), expand = c(0.05,0,0,10)) +
  theme(strip.background = element_blank(), panel.grid = element_blank()) + 
  theme(axis.text = element_text(color = "black"),
        axis.text.x = element_text(angle = 90),
       axis.line = element_line(color = "black", size = 0.5),
        axis.ticks.x = element_blank()) + NoLegend() + ggtitle("Final data all groups")
ncols = length(levels(factor(df4$annotations_manual)))

options(repr.plot.width = 9, repr.plot.height = 7)
p5

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 4)

seurat_meta_data <- treg_subset@meta.data
seurat_meta_data$sample  <- seurat_meta_data$source

# Create grouped dataframe, calculate the frequencies of clusters
df4 <- seurat_meta_data %>% group_by(Patient_ID, seurat_clusters) %>% 
  summarise(n = n()) %>% 
  unique() %>% 
  mutate(freq = n / sum(n)) 


# As we've lost non-grouping variables, let's join them back
md_to_join <- seurat_meta_data %>% dplyr::select(Patient_ID, Disease) %>% unique()
df4  <- left_join(df4, md_to_join, by = "Patient_ID")

# The final plot

df4 %>% 
ggplot(aes(x = Disease, y = freq*100)) + # you can change the x to whatever variable you're interested in
  geom_boxplot(outlier.shape = NA, alpha = 0.3, aes(fill = Disease)) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
geom_beeswarm(size = 3, cex = 3, 
              color = "black", method = "center",
             aes(fill = Disease), shape = 21) +
facet_wrap(~seurat_clusters, scales = "free", ncol = 5) +
   scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
  ylab("Frequency") +
  xlab("Condition") +
#ggpubr::stat_compare_means() +
  ylim(c(0,NA)) + # This ensures that our x axis starts at zero, but feel free to remove this line
  theme_classic() +
  theme(strip.background = element_blank(), panel.grid = element_blank()) + 
ggtheme()



In [ ]:
DimPlot(treg)

In [ ]:
df4$seurat_clusters2  <- recode(df4$seurat_clusters,                                         "0" = "1",
                                                                                               "1" = "2",
                                                                                               "2" = "0",
                                                                                               "3" = "3")

In [ ]:


df4_mean_48  <- df4  %>% group_by(seurat_clusters2, Disease)  %>% 
summarise(median_freq = median(freq)) 


df4_mean_48$dataset  <- "48"

df4_mean_48

In [ ]:
df_cross  <- rbind(df4_mean_hpap, df4_mean_nc, df4_mean_pb, df4_mean_48)

In [ ]:
df_cross

In [ ]:
df_cross  %>% 
mutate(Disease = gsub(Disease, pattern = "T1DM", replacement = "Dia"))  %>% 
ggplot(aes(x = Disease, y = median_freq*100)) + # you can change the x to whatever variable you're interested in
#  geom_boxplot(outlier.shape = NA, alpha = 0.3, aes(fill = Disease)) + 
  geom_dotplot(binaxis='y', stackdir='center', dotsize=0) + 
geom_beeswarm(size = 3, cex = 3, 
              color = "black", method = "center",
             aes(fill = Disease, shape = dataset)) +
geom_line(aes(group = dataset, alpha = 0.1)) +
facet_wrap(~seurat_clusters2, scales = "free", ncol = 5) +
   scale_fill_manual(values = c("#1874cdff",  "#ee6363ff", "#c41515ff","#eeb4b4ff")) +
scale_color_manual(values = c("dodgerblue3",  "#e54c4cff", "#aa2a2aff","#e88989ff")) +
scale_shape_manual(values = c(21,22,24,25)) +
  ylab("Frequency") +
  xlab("Condition") +
#ggpubr::stat_compare_means() +
  ylim(c(0,NA)) + # This ensures that our x axis starts at zero, but feel free to remove this line
  theme_classic() +
  theme(strip.background = element_blank(), panel.grid = element_blank()) + 
ggtheme() + stat_compare_means(label = "p.format", method = "t.test", paired = TRUE)

In [ ]:
ggsave("../figures/treg_validation/all_dataseta_paired_treg_subsets_in_condition.svg",
       create.dir = TRUE,
       width = 20, height = 7, units = "cm")

# Projection of Treg clusters on our Tregs

In [ ]:
DimPlot(treg_nc, group.by = "seurat_clusters")

In [ ]:
treg_nc_cl0  <- subset(treg_nc, seurat_clusters == 0)

In [ ]:
treg_nc_cl0  <- JoinLayers(treg_nc_cl0)

In [ ]:
treg.ref  <- RunUMAP(treg, dims = 1:10, reduction = "pca", return.model = TRUE)

In [ ]:
DimPlot(treg.ref)

## Reference mapping NatCom

In [ ]:
 query  <- treg_nc_cl0

In [ ]:
    treg.anchors <- FindTransferAnchors(reference = treg.ref, query = query,
    dims = 1:10, reference.reduction = "pca", k.score = 5, k.filter = NA, k.anchor = 5)

In [ ]:
treg.anchors

In [ ]:
options(future.globals.maxSize = 65000 * 1024^2)

In [ ]:
query  <- JoinLayers(query)

In [ ]:
query@assays$RNA@layers$scale.data  <- NULL

In [ ]:
query[["RNA"]]  <- as(object = query[["RNA"]], Class = "Assay")

In [ ]:
query

In [ ]:
treg.ref

In [ ]:
treg.ref[["RNA"]]  <- as(object = treg.ref[["RNA"]], Class = "Assay")

In [ ]:
ref <- MapQuery(anchorset = treg.anchors, reference = treg.ref, query = query,
    refdata = treg.ref$seurat_clusters, reference.reduction = "pca", reduction.model = "umap",
               transferdata.args = list(k.weight = 25))

In [ ]:
df <- as.data.frame(treg.ref@reductions$umap@cell.embeddings)
df$source <- "reference"

df2 <- as.data.frame(ref@reductions$ref.umap@cell.embeddings)
df2$source <- "query"

In [ ]:
colnames(df2) <- colnames(df)
df3 <- rbind(df, df2)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)

df3 %>% ggplot() +
  geom_point(aes(x = umap_1, y = umap_2, color = source)) +
  scale_color_manual(values = c("red", "lightgrey")) + 
  theme_classic() + 
  ggtitle("Treg NatCom") + NoLegend()

## Reference mapping ParseBio

In [ ]:
 query  <- treg_pb_cl0

    treg.anchors <- FindTransferAnchors(reference = treg.ref, query = query,
    dims = 1:10, reference.reduction = "pca", k.score = 5, k.filter = NA, k.anchor = 5)

treg.anchors

options(future.globals.maxSize = 65000 * 1024^2)

query  <- JoinLayers(query)

query@assays$RNA@layers$scale.data  <- NULL

query[["RNA"]]  <- as(object = query[["RNA"]], Class = "Assay")

In [ ]:
ref <- MapQuery(anchorset = treg.anchors, reference = treg.ref, query = query,
    refdata = treg.ref$seurat_clusters, reference.reduction = "pca", reduction.model = "umap",
               transferdata.args = list(k.weight = 25))

df <- as.data.frame(treg.ref@reductions$umap@cell.embeddings)
df$source <- "reference"

df2 <- as.data.frame(ref@reductions$ref.umap@cell.embeddings)
df2$source <- "query"

colnames(df2) <- colnames(df)
df3 <- rbind(df, df2)

options(repr.plot.width = 6, repr.plot.height = 5)

df3 %>% ggplot() +
  geom_point(aes(x = umap_1, y = umap_2, color = source)) +
  scale_color_manual(values = c("red", "lightgrey")) + 
  theme_classic() + 
  ggtitle("Treg ParseBio") + NoLegend()

## Reference mapping HPAP

In [ ]:
treg_hpap_spleen_cl0  <- subset(treg_hpap_spleen, seurat_clusters2 == 0)

In [ ]:
 query  <- treg_hpap_spleen_cl0

    treg.anchors <- FindTransferAnchors(reference = treg.ref, query = query,
    dims = 1:10, reference.reduction = "pca", k.score = 5, k.filter = NA, k.anchor = 5)

treg.anchors

options(future.globals.maxSize = 65000 * 1024^2)


query[["RNA"]]  <- as(object = query[["RNA"]], Class = "Assay")

In [ ]:
ref <- MapQuery(anchorset = treg.anchors, reference = treg.ref, query = query,
    refdata = treg.ref$seurat_clusters, reference.reduction = "pca", reduction.model = "umap",
               transferdata.args = list(k.weight = 25))

df <- as.data.frame(treg.ref@reductions$umap@cell.embeddings)
df$source <- "reference"

df2 <- as.data.frame(ref@reductions$ref.umap@cell.embeddings)
df2$source <- "query"

colnames(df2) <- colnames(df)
df3 <- rbind(df, df2)

options(repr.plot.width = 6, repr.plot.height = 5)

df3 %>% ggplot() +
  geom_point(aes(x = umap_1, y = umap_2, color = source)) +
  scale_color_manual(values = c("red", "lightgrey")) + 
  theme_classic() + 
  ggtitle("Treg HPAP") + NoLegend()